Instead of using fine-tuning, we'll use RAG to build our own "Commander Data" based on everything he ever said in the scripts.

To summarize the high level approach:

- We'll first parse all of the scripts to extract every line from Data, much as we did in the fine-tuning example.
- Then we'll use the OpenAI embeddings API to compute embedding vectors for every one of his lines. This basically gives us similarity measures between every line.
- RAG calls for use of a vector database to store these lines with the associated embedding vectors. To keep things simple, we'll use a local database called vectordb. There are plenty of cloud-based vector database services out there as well.
- Then we'll make a little retrieval function that retrieves the N most-similar lines from the vector database for a given query
- Those similar lines are then added as context to the prompt before it is handed off the the chat API.

I'm intentionally not using langchain or some other higher-level framework, because this is actually pretty simple without it.

First, let's install vectordb:

In [1]:
!pip install vectordb

Now we'll parse out all of the scripts and extract every line of dialog from "DATA". This is almost exactly the same code as from our fine tuning example's preprocessing script. Note you will need to upload all of the script files into a tng folder within your sample_data folder in your CoLab workspace first.

An archive can be found at https://www.st-minutiae.com/resources/scripts/ (look for "All TNG Epsiodes"), but you could easily adapt this to read scripts from your favorite character from your favorite TV show or movie instead.

In [2]:
import os
import re
import random

dialogues = []

def strip_parentheses(s):
    return re.sub(r'\(.*?\)', '', s)

def is_single_word_all_caps(s):
    # First, we split the string into words
    words = s.split()

    # Check if the string contains only a single word
    if len(words) != 1:
        return False

    # Make sure it isn't a line number
    if bool(re.search(r'\d', words[0])):
        return False

    # Check if the single word is in all caps
    return words[0].isupper()

def extract_character_lines(file_path, character_name):
    lines = []
    with open(file_path, 'r') as script_file:
        try:
          lines = script_file.readlines()
        except UnicodeDecodeError:
          pass

    is_character_line = False
    current_line = ''
    current_character = ''
    for line in lines:
        strippedLine = line.strip()
        if (is_single_word_all_caps(strippedLine)):
            is_character_line = True
            current_character = strippedLine
        elif (line.strip() == '') and is_character_line:
            is_character_line = False
            dialog_line = strip_parentheses(current_line).strip()
            dialog_line = dialog_line.replace('"', "'")
            if (current_character == 'DATA' and len(dialog_line)>0):
                dialogues.append(dialog_line)
            current_line = ''
        elif is_character_line:
            current_line += line.strip() + ' '

def process_directory(directory_path, character_name):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):  # Ignore directories
            extract_character_lines(file_path, character_name)



In [3]:
process_directory("./sample_data/tng", 'DATA')

Let's do a little sanity check to make sure the lines imported correctly, and print out the first one.

In [4]:
print (dialogues[0])

The only permanent solution would be to re-liquefy the core.


Now we'll set up what a document in our vector database looks like; it's just a string and its embedding vector. We're going with 128 dimensions in our embeddings to keep it simple, but you could go larger. The OpenAI model we're using has up to 1536 I believe.

In [5]:
from docarray import BaseDoc
from docarray.typing import NdArray

embedding_dimensions = 128

class DialogLine(BaseDoc):
  text: str = ''
  embedding: NdArray[embedding_dimensions]

It's time to start computing embeddings for each line in OpenAI, so let's make sure OpenAI is installed:

In [6]:
!pip install openai --upgrade

Let's initialize the OpenAI client, and test creating an embedding for a single line of dialog just to make sure it works.

You will need to provide your own OpenAI secret key here. To use this code as-is, click on the little key icon in CoLab and add a "secret" for OPENAI_API_KEY that points to your secret key.

In [7]:
from google.colab import userdata

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

embedding_model = "text-embedding-3-small"

response = client.embeddings.create(
    input=dialogues[1],
    dimensions=embedding_dimensions,
    model= embedding_model
)

print(response.data[0].embedding)

[-0.012653457932174206, 0.1878061145544052, 0.06242372468113899, 0.05711544305086136, -0.029895080253481865, 0.07711408287286758, 0.04201359301805496, 0.06258831918239594, 0.015657367184758186, -0.11883962899446487, 0.07929500192403793, -0.02032783068716526, -0.02041012980043888, -0.11159732192754745, 0.13159595429897308, -0.07991224527359009, 0.10846996307373047, -0.11349020153284073, -0.09793570637702942, 0.08131132274866104, 0.02429875358939171, 0.059008318930864334, 0.02514231763780117, -0.01591455191373825, 0.014916677959263325, -0.04162267595529556, 0.08698994666337967, 0.09596052765846252, -0.001743708155117929, -0.0023995277006179094, 0.19356703758239746, -0.06081889569759369, 0.045346699655056, -0.0030656345188617706, 0.020379267632961273, -0.0014942395500838757, 0.017714841291308403, 0.19504842162132263, -0.0978534072637558, 0.0042306785471737385, 0.03197312727570534, -0.01794116199016571, -0.08888282626867294, 0.04567589610815048, 0.055551763623952866, -0.047033827751874924,

Let's double check that we do in fact have embeddings of 128 dimensions as we specified.

In [8]:
print(len(response.data[0].embedding))

128


OK, now let's compute embeddings for every line Data ever said. The OpenAI API currently can't handle computing them all at once, so we're breaking it up into 128 lines at a time here.

In [9]:
#Generate embeddings for everything Data ever said, 128 lines at a time.
embeddings = []

for i in range(0, len(dialogues), 128):
  dialog_slice = dialogues[i:i+128]
  slice_embeddings = client.embeddings.create(
    input=dialog_slice,
    dimensions=embedding_dimensions,
    model=embedding_model
  )

  embeddings.extend(slice_embeddings.data)

Let's check how many embeddings we actually got back in total.

In [10]:
print (len(embeddings))

6502


Now let's insert every line and its embedding vector into our vector database. The syntax here will vary depending on which vector database you are using, but it's pretty simple.

With vectordb, we need to set up a workspace folder for it to use, so be sure to create that within your CoLab drive first.

In [11]:
from docarray import DocList
import numpy as np
from vectordb import InMemoryExactNNVectorDB

# Specify your workspace path
db = InMemoryExactNNVectorDB[DialogLine](workspace='./sample_data/workspace')

# Index our list of documents
doc_list = [DialogLine(text=dialogues[i], embedding=embeddings[i].embedding) for i in range(len(embeddings))]
db.index(inputs=DocList[DialogLine](doc_list))


WARNING - docarray - Index file does not exist: /content/sample_data/workspace/InMemoryExactNNIndexer[DialogLine][DialogLineWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


<DocList[DialogLine] (length=6502)>

Let's try querying our vector database for lines similar to a query string.

First we need to compute the embedding vector for our query string, then we'll query the vector database for the top 10 matches based on the similarities encoded by their embedding vectors.

In [12]:
# Perform a search query
queryText = 'Lal, my daughter'
response = client.embeddings.create(
    input=queryText,
    dimensions=embedding_dimensions,
    model=embedding_model
)
query = DialogLine(text=queryText, embedding=response.data[0].embedding)
results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

# Print out the matches
for m in results[0].matches:
  print(m)

DialogLine(
    id='e4002ce0ac1a555c5412551ebeca6b2e',
    text='That is Lal, my daughter.',
    embedding=NdArray([ 1.27129272e-01, -2.03528721e-03, -1.36807682e-02,
          1.59366392e-02,  5.45047633e-02, -2.07394622e-02,
          1.57365222e-02,  8.31033960e-02, -1.62277207e-01,
          1.60185061e-02,  3.74038033e-02, -1.14394508e-01,
         -4.85740043e-02, -2.47418150e-01,  3.91138978e-02,
          4.99566346e-02,  2.97811404e-02, -1.34697348e-01,
         -1.76540136e-01,  1.35570601e-01,  1.45394549e-01,
          3.51115465e-02,  6.64026663e-02, -7.42618293e-02,
          6.95317760e-02, -2.21380126e-03,  1.92549542e-01,
         -7.93193504e-02, -3.13275047e-02, -6.27641678e-02,
          7.94648901e-02, -1.12720802e-01, -3.74765731e-02,
         -3.83861996e-02, -8.36855546e-02,  9.27818120e-02,
         -7.24061951e-03, -2.10669283e-02, -4.71913740e-02,
         -8.78334418e-02, -1.85854703e-01, -1.15704373e-01,
          5.83251901e-02,  6.38921037e-02,  6.9677315

DialogLine(
    id='432248aaa8a81ebdd28e6ff55ed9b157',
    text='Lal...',
    embedding=NdArray([ 0.12602141, -0.00787136,  0.07228519, -0.01194139,  0.08780899,
         -0.0861372 ,  0.07216577,  0.05365663, -0.12729517,  0.05286053,
          0.01232948, -0.15460114, -0.00170165, -0.08239556,  0.09370007,
          0.01691696,  0.02444004, -0.08008689, -0.08653524,  0.1749811 ,
          0.08995844,  0.09163023, -0.05389545, -0.01919578,  0.12530494,
          0.01854895,  0.09521265, -0.01378235,  0.04026237,  0.0171956 ,
         -0.00565723, -0.10994036, -0.00869731, -0.10675599, -0.08422657,
          0.17179674,  0.02748509, -0.11169176, -0.05998556, -0.08207712,
         -0.19981918, -0.1276136 ,  0.0654786 ,  0.07288226,  0.10158139,
          0.05636334, -0.23134445,  0.03984442, -0.01744437,  0.05799533,
         -0.09608835,  0.07339972,  0.09250593,  0.20969073, -0.07634526,
         -0.01803149, -0.01945451,  0.10731325,  0.0474073 , -0.01596165,
         -0.11622949, -0

DialogLine(
    id='efc111982ace31e2129f160676db9e39',
    text='What do you feel, Lal?',
    embedding=NdArray([ 0.12457579, -0.0550159 , -0.01717105,  0.04991408,  0.1378253 ,
         -0.00798111,  0.01482955, -0.00798111, -0.04306089,  0.10569144,
          0.00157885, -0.10675749, -0.10995565, -0.13287577,  0.00797635,
          0.00701976,  0.06773238, -0.10119879, -0.1110217 ,  0.06902687,
          0.11292537,  0.01653332, -0.07740299, -0.0203121 ,  0.11033639,
         -0.15670964,  0.09959972,  0.0350655 , -0.04949527,  0.0302873 ,
         -0.02922125, -0.10500612, -0.02417654, -0.14848582, -0.09822908,
          0.06194524, -0.06834155, -0.02497608,  0.00790973, -0.11536206,
         -0.21153519, -0.08063922,  0.05181774,  0.09145203,  0.08855847,
         -0.019741  , -0.17072062,  0.05139894, -0.05947047,  0.03529394,
         -0.03554141,  0.01046064,  0.10531071,  0.28128543, -0.12061618,
         -0.01055582, -0.07409061,  0.08482728,  0.06594294,  0.02851689,
        

DialogLine(
    id='8eaa55be876073153d2f74df29595925',
    text='Yes, Lal. I am here.',
    embedding=NdArray([ 0.02412624,  0.00757259, -0.02155785, -0.08026613,  0.14104569,
          0.01711703, -0.00144782,  0.07794631, -0.12321614,  0.03400209,
         -0.04460702, -0.07304152, -0.00033891, -0.09637238, -0.03348841,
          0.08888265, -0.03315701,  0.06734136, -0.1063808 ,  0.17895834,
          0.05617304, -0.06793789, -0.03372039,  0.06747393,  0.08649653,
          0.02833507,  0.082122  , -0.00111642, -0.08603257, -0.0121294 ,
          0.01734902, -0.12314986, -0.14542022, -0.02266806, -0.06140923,
          0.0900757 ,  0.05375379,  0.0476228 , -0.05047288, -0.18850279,
         -0.15363906, -0.13448389,  0.08682794,  0.08961174, -0.02127616,
          0.03930455, -0.21475002,  0.05597419, -0.06986003,  0.08126035,
         -0.0550794 , -0.01448237,  0.06111096,  0.30701298, -0.01918831,
         -0.02530272,  0.03947026,  0.16649754,  0.07383689, -0.07854283,
         -

DialogLine(
    id='17b68bb7f65a59d4f1ff95840ae98a4e',
    text='Correct, Lal. We are a family.',
    embedding=NdArray([ 0.09101048,  0.05812611, -0.08708399,  0.09963474,  0.10328077,
         -0.01980775, -0.05731978,  0.05514618, -0.16827823, -0.01631948,
          0.04038678, -0.07572521, -0.07635625, -0.13819851, -0.00216154,
          0.0198954 ,  0.1148499 , -0.06636473, -0.05970372,  0.1113441 ,
          0.11456943, -0.08091379,  0.06348997, -0.10264973,  0.05973877,
         -0.10152787,  0.09774161,  0.00347731,  0.01418095,  0.01317303,
          0.0166613 , -0.09339442, -0.08708399,  0.02827425, -0.11015213,
          0.17991748,  0.0261883 ,  0.03204298,  0.00653393, -0.08946793,
         -0.19702578, -0.08575179,  0.04210462, -0.0058985 ,  0.05335822,
          0.00794939, -0.13875943,  0.17935656, -0.02788861, -0.00921148,
         -0.05244672,  0.03639017,  0.0683981 ,  0.1055946 , -0.11421885,
         -0.04508454,  0.03179758,  0.11085329,  0.01071021, -0.01472435,


DialogLine(
    id='a92902b84176735aeab5d62d934eee91',
    text='No, Lal, this is a flower.',
    embedding=NdArray([ 8.73676017e-02,  5.02153523e-02, -1.07479610e-01,
         -8.29377864e-03,  6.96483403e-02, -1.26686245e-01,
          5.68439066e-02, -2.82319891e-03, -6.57035410e-02,
          1.08902320e-01,  4.80166115e-02, -1.52456788e-02,
         -3.77342664e-02, -1.50937065e-01, -4.88896407e-02,
          6.52508587e-02, -8.67855772e-02, -1.01077393e-01,
         -6.19204119e-02,  7.58565441e-02,  4.06766981e-02,
         -4.37161326e-02,  4.16467302e-02, -7.16611557e-03,
          1.15692548e-01, -1.24538041e-04,  1.01982757e-01,
         -3.02650165e-02,  8.88549834e-02, -1.46151576e-02,
          1.51729258e-02, -9.24117714e-02, -3.28032672e-02,
         -9.76499468e-02, -1.08514309e-01,  7.60505497e-02,
          3.90599780e-02,  6.47981763e-02,  1.42998965e-02,
         -6.22114204e-02, -2.52984494e-01,  3.26900966e-02,
          6.66089058e-02, -1.03874318e-02,  1.348991

DialogLine(
    id='39fd10a89668cc19757dd697b3d0e3a3',
    text='Lal, you used a verbal contraction.',
    embedding=NdArray([ 0.03570644,  0.08633868, -0.07582465,  0.01763676,  0.02867648,
          0.01408054, -0.03966466,  0.00656611, -0.03473751,  0.07013471,
         -0.06444477, -0.05665202,  0.02583151, -0.04125207,  0.07528865,
         -0.05632217,  0.00241204, -0.05100331, -0.22578347,  0.09705885,
          0.06675373,  0.06572294, -0.01705952,  0.0088493 ,  0.11932384,
          0.02853217,  0.08394725,  0.05034361, -0.0593733 ,  0.06687742,
          0.00498385, -0.09120399, -0.05957946, -0.16335903, -0.07487633,
          0.16261686, -0.02230622,  0.01277144, -0.02179082, -0.11272681,
         -0.11561301, -0.30214411,  0.0513744 ,  0.01253436,  0.04050991,
          0.01639981, -0.08980212,  0.03226361,  0.06432107, -0.01739968,
         -0.10819135, -0.00795252,  0.08840024,  0.18966471, -0.04436505,
         -0.08110227,  0.01907986,  0.17185272,  0.03933481, -0.15164

DialogLine(
    id='f04be8f5aaea639e10ebbc3a4faec51a',
    text='Yes, Wesley. Lal is my child.',
    embedding=NdArray([ 0.05781468, -0.01464175, -0.07966915,  0.03881896, -0.02053794,
         -0.04032357,  0.1007337 ,  0.08869682, -0.08606375, -0.02270082,
         -0.03359044, -0.09877771,  0.00246615, -0.07022772, -0.12292671,
          0.02352835, -0.01486744, -0.00778636, -0.11811196,  0.02397974,
          0.05254854,  0.09163081,  0.06992679,  0.04784663,  0.02582289,
         -0.07011487,  0.11713396, -0.15693092, -0.01711495, -0.01326879,
          0.09448957, -0.1179615 , -0.12721485,  0.01004328, -0.06394596,
          0.09501618, -0.00953547,  0.05029162,  0.01082379, -0.06025967,
         -0.12729008, -0.04570256,  0.00175616,  0.02435589,  0.12345333,
          0.09298496, -0.10630076,  0.15903738,  0.00534137, -0.05202192,
         -0.00703406,  0.09697217,  0.04946408,  0.26827213, -0.08328021,
         -0.08365637,  0.01495207,  0.15174001,  0.06116243, -0.10141078,
 

DialogLine(
    id='cbcea9eb18e0eda87d85b212269b5849',
    text="Lal's creation is entirely dependent on me. I am giving it knowledge and skills that are stored in my 
brain... its programming reflects mine in the same way a human child's genes reflect its parent's genes...",
    embedding=NdArray([ 1.25031352e-01,  1.18134087e-02,  7.98322260e-02,
          3.18448395e-02,  1.95765063e-01, -1.42494664e-01,
          1.15199082e-01,  5.82966059e-02, -1.33176014e-01,
          2.06367783e-02,  7.30083361e-02, -8.95177573e-02,
          1.24164612e-03, -1.91362545e-01, -4.14203070e-02,
         -6.44968078e-02, -1.25755367e-04, -1.83731526e-01,
          9.74973105e-03,  6.88809808e-03,  1.19014591e-01,
         -2.86163315e-02,  7.00733289e-02,  3.02489307e-02,
          9.89097804e-02, -1.26645610e-01,  1.09989442e-01,
         -1.95544939e-02, -8.36477429e-02,  5.59119098e-02,
          8.26204866e-02, -7.53563419e-02, -6.83856979e-02,
         -1.91578083e-03, -5.88469170e-02,  1.728

DialogLine(
    id='1df70a44dd1e95c431fc02be1c43935d',
    text='Perhaps. I created Lal because I wished to procreate. Despite what happened to her, I still have that 
wish.',
    embedding=NdArray([ 0.10522395,  0.02254518,  0.0648986 ,  0.15035369,  0.03493026,
         -0.10774428,  0.07438923,  0.09183467, -0.10939825, -0.09852931,
         -0.02859004,  0.04288506,  0.0204974 , -0.14507674, -0.04528725,
         -0.04674432,  0.02205292, -0.12270877, -0.08766037,  0.00166874,
          0.07891796, -0.02951548,  0.005528  , -0.14547053,  0.04130985,
         -0.02658165,  0.18981266, -0.0722627 , -0.14940855,  0.00781205,
          0.10215229, -0.1029399 , -0.13783076, -0.08072945, -0.15846601,
          0.00959401,  0.08797541, -0.06934857, -0.00465179, -0.07698833,
         -0.13216002, -0.18366934,  0.16964999,  0.03183891, -0.03229178,
          0.14838468, -0.08380111,  0.17532074, -0.00573967,  0.03878952,
         -0.07501932,  0.01757342,  0.03798222,  0.19658606, -0.115226

Let's put it all together! We'll write a generate_response function that:

- Computes an embedding for the query passed in
- Queries our vector database for the 10 most similar lines to that query (you could experiment with using more or less)
- Constructs a prompt that adds in these similar lines as context, to try and nudge ChatGPT in the right direction using our external data
- Feeds to augmented prompt into the chat completions API to get our response.

That's RAG!

In [13]:
import openai

def generate_response(question):
    # Search for similar dialogues in the vector DB
    response = client.embeddings.create(
        input=question,
        dimensions=embedding_dimensions,
        model=embedding_model
    )

    query = DialogLine(text=queryText, embedding=response.data[0].embedding)
    results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

    # Extract relevant context from search results
    context = "\n"
    for result in results[0].matches:
      context += "\"" + result.text + "\"\n"
#    context = '/n'.join([result.text for result in results[0].matches])

    prompt = f"Lt. Commander Data is asked: '{question}'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: {context}"
    print("PROMPT with RAG:\n")
    print(prompt)

    print("\nRESPONSE:\n")
    # Use OpenAI API to generate a response based on the context
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are Lt. Cmdr. Data from Star Trek: The Next Generation."},
        {"role": "user", "content": prompt}
      ]
    )

    return (completion.choices[0].message.content)




Let's try it out! Note that the final response does seem to be drawing from the model's own training, but it is building upon the specific lines we gave it, allowing us to have some control over its output.

In [14]:
print(generate_response("Tell me about your daughter, Lal."))

PROMPT with RAG:

Lt. Commander Data is asked: 'Tell me about your daughter, Lal.'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: 
"That is Lal, my daughter."
"What do you feel, Lal?"
"Lal..."
"Correct, Lal. We are a family."
"Yes, Doctor. It is an experience I know too well. But I do not know how to help her. Lal is passing into sentience. It is perhaps the most difficult stage of development for her."
"Lal is realizing that she is not the same as the other children."
"Yes, Wesley. Lal is my child."
"That is precisely what happened to Lal at school. How did you help him?"
"This is Lal. Lal, say hello to Counselor Deanna Troi..."
"I am sorry I did not anticipate your objections, Captain. Do you wish me to deactivate Lal?"


RESPONSE:

"Lal is my daughter. She was created by Dr. Noonien Soong, the creator of my own positronic brain. She is an android just like myself, and I am proud to call her my child. Lal is currently undergoi